In [17]:
import json
import pandas as pd
from itertools import combinations
import Levenshtein
import numpy as np
from sklearn.cluster import DBSCAN

In [2]:
with open('data/przepisy.json') as f:
    recipes = json.load(f)


In [3]:
ingredients = []
for recipe in recipes:
    for ingredient in recipe['ingredient_list']:
        ingredients.append([ recipe['url'], ingredient['ingredient_name'], 
                            ingredient['ingredient_amount'],
                            ingredient['ingredient_unit']])

In [4]:
df = pd.DataFrame(ingredients, columns=["URL", "nazwa", "ilosc", "jednostka"])

In [5]:
df["nazwa"].head()

0     makaron świderki
1    filet z kurczaka 
2               cebula
3              papryka
4              pomidor
Name: nazwa, dtype: object

In [69]:
names = df["nazwa"].unique()

In [70]:
pairs = combinations(enumerate(names), 2)
n_names = len(names)
distances = np.zeros((n_names, n_names))
for (index1, prod1), (index2, prod2) in pairs:
    distance = Levenshtein.distance(prod1, prod2) / max(len(prod1), len(prod2))
    distances[index1, index2] = distance
    distances[index2, index1] = distance

In [65]:
distances

array([[0.        , 0.88235294, 1.        , ..., 0.73076923, 0.9047619 ,
        0.6875    ],
       [0.88235294, 0.        , 0.82352941, ..., 0.80769231, 0.80952381,
        0.76470588],
       [1.        , 0.82352941, 0.        , ..., 0.92307692, 0.80952381,
        0.92857143],
       ...,
       [0.73076923, 0.80769231, 0.92307692, ..., 0.        , 0.80769231,
        0.80769231],
       [0.9047619 , 0.80952381, 0.80952381, ..., 0.80769231, 0.        ,
        0.76190476],
       [0.6875    , 0.76470588, 0.92857143, ..., 0.80769231, 0.76190476,
        0.        ]])

In [71]:
clust = DBSCAN(eps=0.3, min_samples=2, metric='precomputed', n_jobs=-1)
clust.fit(distances)

DBSCAN(algorithm='auto', eps=0.3, leaf_size=30, metric='precomputed',
       metric_params=None, min_samples=2, n_jobs=-1, p=None)

In [72]:
labels = clust.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
clusters = clust.labels_
for cluster_id in range(n_clusters):
    indexes, = np.where(labels == cluster_id)
    cluster_names = [names[i] for i in indexes.tolist()]
    print(set(cluster_names), len(cluster_names))

{'makaron sojowy ', 'makaron gwiazdki', 'makaron literki', 'makaron rurki', 'makaron gryczany', 'makaron typ muszelki', 'makaron uszka', 'makaron krótki, np. świderki', 'makaron muszelki', 'makaron np. spaghetti', 'makaron duże rurki', 'makaron orechiette', 'makaron pennette', 'makaron gniazda nitki', 'makaron nitki', 'gotowy makaron cannelloni', 'makaron macaroni', 'makaron orkiszowy świderki', 'makaronowe rurki', 'makaron koraliki', 'makaron orecchiette ', 'makaron spagetti', 'makaron duże muszle', 'makaron gniazda ', 'makaron (duże muszle)', 'makaron drobny', 'makaron razowy', 'makaron Rigatoni', 'makaron, np. penne', 'makaron, np. rurki', 'makaron spaghetti', 'makaron diatali', 'makaron fusilli', 'makaron cannelloni', 'makaron ryzowy', 'makaron tortellini', 'makaron pipette', 'makaron penne', 'makaron, np. świderki', 'makaron np. świderki', 'makaron rigatoni', 'makaron wstążki', 'makaron chiński', 'makaron kokardki', 'makaron świderki', 'makaron orzo ', 'makaron Bucatini', 'makaron

{'wino czerwone wytrawne', 'wino czerwone półwytrawne'} 2
{'dynia hokkaido', 'dynia np. Hokkaido', 'dynia, np. hokkaido', 'mała dynia hokkaido'} 4
{'gicze cielęce ', 'gicz cielęca', 'pieczeń cielęca'} 3
{'ubite jajka', 'ubite jajko'} 2
{'ryba biała o zwartym mięsie', 'ryba o zwartym mięsie', 'biała ryba o zwartym mięsie '} 3
{'steki z karkówki ', 'kotlety z karkówki '} 2
{'gnocchi:', 'Gnocchi'} 2
{'sos słodko-kwaśny', 'sos słodko-pikantny'} 2
{'orzechy nerkowca niesolone', 'orzechy nerkowca (namoczone)', 'orzechy nerkowca lub ziemne'} 3
{'warzywa suszone', 'warzywa sezonowe'} 2
{'miąższ z dyni ', 'miąższ dyni'} 2
{'seler (ugotowany)', 'seler ugotowany'} 2
{'szczypta majeranku', 'szczypta szafranu', 'szczypta cząbru'} 3
{'kiełki brokuła ', 'kiełki brokułów'} 2
{'filet z halibuta', 'stek z halibuta'} 2
{'Wieprzowina bez kości (kark, łopatka, szynka)', 'Wieprzowina bez kości ( kark łopatka, szynka)', 'Wieprzowina bez kości (np. łopatka)'} 3
{'kawałek filetu z pstrąga', 'filet z pstrąga', 